In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras as keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [49]:
fermo = pd.read_csv("fermo.csv")
mov = pd.read_csv("mov.csv")

mov



ParserError: Error tokenizing data. C error: Expected 11 fields in line 3, saw 12


In [43]:
x_train, x_test, y_train, y_test = train_test_split(data.loc[:, 'xa'], test_size=0.2)

ValueError: not enough values to unpack (expected 4, got 2)

# Modello

In [36]:
model = keras.Sequential([
    layers.Dense(units = 256, input_shape=[11], activation='relu'),
    layers.Dropout(rate=0.3),
    layers.BatchNormalization(),
    layers.Dense(units = 256, activation = 'relu'),
    layers.Dropout(rate=0.3),
    layers.BatchNormalization(),
    layers.Dense(units = 256, activation = 'relu'),
    layers.Dropout(rate=0.3),
    layers.BatchNormalization(),
    layers.Dense(units = 1, activation = 'sigmoid')
])

In [35]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

In [37]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

In [ ]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=512,
    epochs=100,
    callbacks=[early_stopping],
)